In [18]:
import pandas as pd
import re
df=pd.read_csv(r"E:\LLMOPs\aiops\data\raw\arxiv_subset.csv")

In [19]:
df.head(3)

,id,categories,text
0,807.4728,gr-qc,On the dynamics of thin shells of counter rota...
1,808.4052,math.OA,Measured quantum groupoids with a central basi...
2,708.2013,gr-qc,Quantum amplitudes in black-hole evaporation: ...


In [20]:
df["categories"].unique()

array(['gr-qc', 'math.OA', 'physics.gen-ph', ...,
       'math.NA physics.comp-ph stat.CO', 'cs.DL astro-ph',
       'math.CV math.HO'], shape=(2296,), dtype=object)

In [21]:
df['categories'].value_counts()

categories
astro-ph                                    3318
hep-ph                                      1069
quant-ph                                     894
hep-th                                       746
cond-mat.mtrl-sci                            436
                                            ... 
cond-mat.str-el cond-mat.supr-con hep-th       1
math.CA math-ph math.MP math.PR                1
physics.flu-dyn cond-mat.stat-mech             1
nlin.CD astro-ph physics.flu-dyn               1
physics.bio-ph q-bio.PE                        1
Name: count, Length: 2296, dtype: int64

In [22]:
df["categories"].nunique()

2296

In [23]:
df["text"][0]

'On the dynamics of thin shells of counter rotating particles. In this paper we study the dynamics of self gravitating spherically symmetric\nthin shells of counter rotating particles. We consider all possible velocity\ndistributions for the particles, and show that the equations of motion by\nthemselves do not constrain this distribution. We therefore consider the\ndynamical stability of the resulting configurations under several possible\nprocesses. This include the stability of static configurations as a whole,\nwhere we find a lower bound for the compactness of the shells. We analyze also\nthe stability of the single particle orbits, and find conditions for "single\nparticle evaporation". Finally, in the case of a shell with particles whose\nangular momenta are restricted to two values, we consider the conditions for\nstability under splitting into two separate shells. This analysis leads to the\nconclusion that under certain conditions, that are given explicitly, an\nevolving shel

In [24]:
df["text"][2]

"Quantum amplitudes in black-hole evaporation: Spins 1 and 2. Quantum amplitudes for $s=1$ at Maxwell fields and for $s=2$ linearised\ngravitational wave perturbations of a spherically symmetric Einstein/massless\nscalar background, describing gravitational collapse to a black hole, are\ntreated by analogy with a previous treatment of $s=0$ scalar-field\nperturbations of gravitational collapse at late times. In both the $s=1$ and\n$s=2$ cases, we isolate suitable 'co-ordinate' variables which can be taken as\nboundary data on a final space-like hypersurface $\\Sigma_F$. For simplicity, we\ntake the data on an initial pre-collapse surface $\\Sigma_I$ to be exactly\nspherically symmetric. The (large) Lorentzian proper-time interval between\n$\\Sigma_{I}, \\Sigma_{F}$, measured at spatial infinity, is denoted by $T$. The\ncomplexified classical boundary-value problem is expected to be well-posed,\nprovide that the time interval $T$ has been rotated into the complex:\n$T\\to{\\mid}T{\\mid}

In [25]:
def remove_special_symbols(x):
    if isinstance(x,str):
        text=x.replace('\n',' ')
        text=re.sub(r'\s+',' ',text).strip()
        return text
    else:
        return text


In [26]:
df["text"]=df["text"].apply(remove_special_symbols)

In [27]:
df.head(3)

,id,categories,text
0,807.4728,gr-qc,On the dynamics of thin shells of counter rota...
1,808.4052,math.OA,Measured quantum groupoids with a central basi...
2,708.2013,gr-qc,Quantum amplitudes in black-hole evaporation: ...


In [28]:
df["text"][2]

"Quantum amplitudes in black-hole evaporation: Spins 1 and 2. Quantum amplitudes for $s=1$ at Maxwell fields and for $s=2$ linearised gravitational wave perturbations of a spherically symmetric Einstein/massless scalar background, describing gravitational collapse to a black hole, are treated by analogy with a previous treatment of $s=0$ scalar-field perturbations of gravitational collapse at late times. In both the $s=1$ and $s=2$ cases, we isolate suitable 'co-ordinate' variables which can be taken as boundary data on a final space-like hypersurface $\\Sigma_F$. For simplicity, we take the data on an initial pre-collapse surface $\\Sigma_I$ to be exactly spherically symmetric. The (large) Lorentzian proper-time interval between $\\Sigma_{I}, \\Sigma_{F}$, measured at spatial infinity, is denoted by $T$. The complexified classical boundary-value problem is expected to be well-posed, provide that the time interval $T$ has been rotated into the complex: $T\\to{\\mid}T{\\mid}\\exp(-i\\th

In [29]:
import re

def super_clean_scientific_text_perfect(text):
    if not isinstance(text, str):
        return text
    
    # 1. Isolate and Store LaTeX Formulas
    formulas = re.findall(r'\$.*?\$', text) 
    placeholder = "__FORMULA__"
    
    # Replace formulas with a bare placeholder
    text_with_placeholders = re.sub(r'\$.*?\$', placeholder, text)
    
    # 2. General Cleaning on the Text *excluding* formulas
    cleaned_text_only = text_with_placeholders.replace("\n", ' ')
    
    # Remove complex symbols from surrounding text, KEEPING basic punctuation
    # Also, the previous output showed 'qpi/2' instead of 'pi/2'. Let's ensure 'pi' is handled.
    cleaned_text_only = re.sub(r'[^\w\s.,:;?]', '', cleaned_text_only)
    cleaned_text_only = re.sub(r'\s+', ' ', cleaned_text_only).strip()
    
    # 3. Dynamic Simplification of Formulas
    simplified_formulas = []
    
    for f in formulas:
        f_content = f.strip('$')
        
        # Replace common structural characters
        f_content = f_content.replace(r'\over', '/')
        f_content = f_content.replace(r'\to', '->')
        f_content = f_content.replace(r'\mid', '|')
        f_content = f_content.replace(r'\cdot', '*')
        f_content = f_content.replace(r'\exp', 'exp')
        f_content = f_content.replace(r'\rm', '')
        f_content = f_content.replace(r'\le', '<=')
        f_content = f_content.replace(r'\pi', 'pi') # Fixes the common pi symbol
        
        # Generalized removal of LaTeX structure
        f_content = f_content.replace('\\', '')
        f_content = f_content.replace('{', '')
        f_content = f_content.replace('}', '')
        
        # Clean up any extra spaces introduced during the cleanup
        f_content = re.sub(r'\s+', '', f_content).strip()
        
        # Re-add the dollar signs
        simplified_formulas.append(f"${f_content}$")
    
    # 4. Reconstruct the final text - FIXING SPACING
    
    final_text = cleaned_text_only
    
    # Use a loop to replace the placeholders with the formula, ensuring spaces are added
    for formula in simplified_formulas:
        # Replace the first instance of the placeholder with the formula surrounded by spaces
        # Then, clean up any double spaces that this might create (e.g., if placeholder was next to an existing space)
        final_text = final_text.replace(placeholder, f" {formula} ", 1)

    # Final cleanup of multiple spaces and leading/trailing spaces
    return re.sub(r'\s+', ' ', final_text).strip()

In [30]:
df["text"] = df["text"].apply(super_clean_scientific_text_perfect)

In [31]:
df["text"][2]

'Quantum amplitudes in blackhole evaporation: Spins 1 and 2. Quantum amplitudes for $s=1$ at Maxwell fields and for $s=2$ linearised gravitational wave perturbations of a spherically symmetric Einsteinmassless scalar background, describing gravitational collapse to a black hole, are treated by analogy with a previous treatment of $s=0$ scalarfield perturbations of gravitational collapse at late times. In both the $s=1$ and $s=2$ cases, we isolate suitable coordinate variables which can be taken as boundary data on a final spacelike hypersurface $Sigma_F$ . For simplicity, we take the data on an initial precollapse surface $Sigma_I$ to be exactly spherically symmetric. The large Lorentzian propertime interval between $Sigma_I,Sigma_F$ , measured at spatial infinity, is denoted by $T$ . The complexified classical boundaryvalue problem is expected to be wellposed, provide that the time interval $T$ has been rotated into the complex: $T->|T|exp(-itheta)$ , for $0<theta<=qpi/2$ . We calcula

In [32]:
df['text']=df['text'].str.lower()

In [ ]:
df=df.sample(1000)

In [33]:
df.to_csv("E:/LLMOPs/aiops/data/processed/arxiv_processed.csv", index=False)


# Data for the finetuning


In [103]:
from sklearn.model_selection import train_test_split

train_df, val_df = train_test_split(df, test_size=0.1, random_state=42)

train_df.to_json("E:/LLMOPs/aiops/data/train.jsonl", orient="records", lines=True)
val_df.to_json("E:/LLMOPs/aiops/data/val.jsonl", orient="records", lines=True)
